In [2]:
import pandas as pd
import numpy as np
import gc

In [3]:
df_events = pd.read_csv('./vi_data/events_train.csv')
df_purchases = pd.read_csv('./vi_data/purchases_train.csv')

/usr/local/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


# Nápady

https://heartbeat.fritz.ai/recommender-systems-with-python-part-i-content-based-filtering-5df4940bd831
https://towardsdatascience.com/prototyping-a-recommender-system-step-by-step-part-1-knn-item-based-collaborative-filtering-637969614ea
https://www.youtube.com/watch?v=Ty3Ui_mQtxY

# Prehľad dát

In [4]:
df_events.head()

,customer_id,timestamp,event_type,product_id,title,category_name,price
0,204685f9-00f0-4a36-bab1-e0b74673c3b1,2019-07-01 00:00:01.101977 UTC,view_item,f130c542-d793-41d0-88f6-d1c52b4fe8f0,Michael Kors Nappa Leather Racer Jacket,NaN,747.49
1,552445a5-f41d-4b25-b479-4e776d9bee61,2019-07-01 00:00:01.901589 UTC,view_item,29db8f57-49bd-4108-a0bb-0d245278e999,Puma Woven Shorts Girls,NaN,11.92
2,fac01f48-a03a-41f2-acd5-b5f4e407d9c2,2019-07-01 00:00:08.954928 UTC,view_item,a90bddc4-8eb7-4c99-aa4f-2f2377c246a2,Trendyol Multi-colored Flywheel Detailed Swimwear,NaN,23.85
3,552445a5-f41d-4b25-b479-4e776d9bee61,2019-07-01 00:00:09.721589 UTC,view_item,600268e1-a2ba-4b1b-ab57-58c577412af0,Puma Woven Shorts Girls,NaN,11.92
4,bc59a341-040f-41ea-8f33-cc4a02c2d85d,2019-07-01 00:00:13.745257 UTC,view_item,b74e0b38-1efe-4d86-9718-952cc2580212,Women's dress ROXY NEW LEASEOFLIFE J KTDR,NaN,40.00


In [5]:
df_purchases.head()

,customer_id,timestamp,event_type,product_id,title,category_name,price
0,0d1b7397-7d3c-44c0-9efc-d38bf197828b,2019-07-01 00:05:54.308966 UTC,purchase_item,954f9f2c-d3ca-4236-ac9a-4ea7bcf09305,Trendyol Leopard print Bikini bottom,NaN,NaN
1,81ccae7e-e496-4997-a289-4669bf53f33e,2019-07-01 00:20:03.404186 UTC,purchase_item,75b281e5-8a16-42cb-9ae0-9a98db7a2c40,No Fear Reflect Shorts Mens,NaN,NaN
2,50777c55-8dd6-4309-a5ca-26e66c8a8279,2019-07-01 00:34:35.989935 UTC,purchase_item,0112dec8-47f5-4c2c-9109-571e2dbb6345,Trendyol Multicolor camisole Blouse,NaN,NaN
3,50777c55-8dd6-4309-a5ca-26e66c8a8279,2019-07-01 00:34:35.991935 UTC,purchase_item,0fa25a2d-2aa1-4397-82f1-5a64f3b1272d,Trendyol White High Waist Skinny Jeans,NaN,NaN
4,647d269f-b18f-4558-8653-93369d862ec9,2019-07-01 00:52:53.083698 UTC,purchase_item,81c01216-55a9-4588-a722-bccf0bf35fd5,Firetrap Lazer Polo Shirt,NaN,NaN


## Informácie o dátach

In [6]:
df_events.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14614385 entries, 0 to 14614384
Data columns (total 7 columns):
customer_id      object
timestamp        object
event_type       object
product_id       object
title            object
category_name    object
price            float64
dtypes: float64(1), object(6)
memory usage: 780.5+ MB


In [7]:
df_purchases.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 188712 entries, 0 to 188711
Data columns (total 7 columns):
customer_id      188712 non-null object
timestamp        188712 non-null object
event_type       188712 non-null object
product_id       188712 non-null object
title            188684 non-null object
category_name    143814 non-null object
price            143826 non-null float64
dtypes: float64(1), object(6)
memory usage: 10.1+ MB


In [8]:
# sort data by date (timestamp)
df_events.sort_values(by=['timestamp'], inplace=True, ascending=False)
df_purchases.sort_values(by=['timestamp'], inplace=True, ascending=False)

# Predspracovanie dát

In [9]:
from sklearn import preprocessing

In [10]:
# spojenie dát
df = pd.concat([df_events, df_purchases])

del [[df_events, df_purchases]]
gc.collect()

0

In [11]:
df.drop(['price'], axis=1, inplace=True)

In [12]:
df['event_type'] = df['event_type'].replace({ 'view_item': 1, 'purchase_item': 2 })

In [13]:
prepared_df = df.groupby(['customer_id', 'product_id']).sum().reset_index().sort_values(by=['event_type'], ascending=False)

del [[df]]
gc.collect()

0

# Zakódovanie idčiek

In [14]:
from sklearn import preprocessing

In [15]:
le_customer = preprocessing.LabelEncoder()
le_product = preprocessing.LabelEncoder()

prepared_df['customer_id'] = le_customer.fit_transform(prepared_df['customer_id'])
prepared_df['product_id'] = le_product.fit_transform(prepared_df['product_id'])

In [16]:
prepared_df.head()

,customer_id,product_id,event_type
9247622,2138502,10450,550
3614166,836610,7499,460
2785564,644747,6304,444
6704514,1549078,7499,435
5116900,1184349,76248,435


# Rozdelenie na trénovaciu a testovaciu zložku

In [17]:
from sklearn.model_selection import train_test_split

In [18]:
df_train, df_test = train_test_split(prepared_df, test_size=0.99, shuffle=False)

del [[prepared_df]]
gc.collect()

0

In [19]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 95659 entries, 9247622 to 5344672
Data columns (total 3 columns):
customer_id    95659 non-null int64
product_id     95659 non-null int64
event_type     95659 non-null int64
dtypes: int64(3)
memory usage: 2.9 MB


In [20]:
df_test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9470255 entries, 130573 to 9565913
Data columns (total 3 columns):
customer_id    int64
product_id     int64
event_type     int64
dtypes: int64(3)
memory usage: 289.0 MB


In [21]:
df_test

,customer_id,product_id,event_type
130573,30272,114934,7
8888237,2055495,46021,7
4953448,1146289,63415,7
3095417,717199,81732,7
3309484,766378,95239,7
...,...,...,...
3707778,858422,108621,1
3707777,858422,108416,1
3707776,858422,108143,1
3707775,858422,106772,1


# Prehľad trénovacích a testovacích dát

In [22]:
df_test.head()

,customer_id,product_id,event_type
130573,30272,114934,7
8888237,2055495,46021,7
4953448,1146289,63415,7
3095417,717199,81732,7
3309484,766378,95239,7


In [23]:
df_train.head()

,customer_id,product_id,event_type
9247622,2138502,10450,550
3614166,836610,7499,460
2785564,644747,6304,444
6704514,1549078,7499,435
5116900,1184349,76248,435


# Tvorba sparse matrix

robím podla tohoto
https://towardsdatascience.com/prototyping-a-recommender-system-step-by-step-part-1-knn-item-based-collaborative-filtering-637969614ea

Ktorú sparse_matrix použiť:
https://cmdlinetips.com/2018/03/sparse-matrices-in-python-with-scipy/

In [24]:
from scipy.sparse import coo_matrix

In [25]:
mat_coo = coo_matrix(
    (
        np.array(df_train['event_type']), 
        (
            np.array(df_train['product_id']), 
            np.array(df_train['customer_id'])
        )
    )
)

In [26]:
mat_csr = mat_coo.tocsr(copy=False)

In [27]:
mat_csr.shape

(123837, 2211272)

# Odporúčanie

In [28]:
from sklearn.neighbors import NearestNeighbors

In [30]:
model_knn = NearestNeighbors(metric='cosine', algorithm='brute', n_neighbors=20, n_jobs=-1)
model_knn.fit(mat_csr)

NearestNeighbors(algorithm='brute', leaf_size=30, metric='cosine',
                 metric_params=None, n_jobs=-1, n_neighbors=20, p=2,
                 radius=1.0)

In [39]:
def recommend(model_knn, data, index, n_recos): 
    distances, indices = model_knn.kneighbors(data[index], n_neighbors=n_recos + 1)
    
    product_ids = le_product.inverse_transform(indices[0])
    
    return distances[0], product_ids

In [44]:
distances, product_ids = recommend(model_knn, mat_csr, 95239, 10)

for distance, product_id in zip(distances, product_ids):
    print(f"Recommended item: {product_id} with distance {distance}.")

Recommended item: c552587b-cb1e-4698-a101-3c8a4762b40a with distance 3.3306690738754696e-16.
Recommended item: 9480b05d-dd2f-4104-ac8a-ef3c1767a975 with distance 0.8292518940100772.
Recommended item: 8460e0f9-d46f-421d-8939-6d47f02dc7b1 with distance 0.8812737486722363.
Recommended item: 31ae211d-57a9-4bfa-b95f-40776af03c3f with distance 0.8820942356738127.
Recommended item: ca2cb8ca-595d-48e1-aea2-c53e08f73ab4 with distance 0.8942370121273663.
Recommended item: de1b9182-defe-4c9f-8111-18fb8f1fa6c5 with distance 0.9022970710565205.
Recommended item: 33402bf2-b5b2-417e-aff1-fca439e18505 with distance 0.9067305085106581.
Recommended item: 67034a76-e06f-416c-ad25-2912593b10f0 with distance 0.9109553115041773.
Recommended item: 9d185422-4597-4ee8-8910-0e5913e22162 with distance 0.9109553115041773.
Recommended item: 6f5c4a48-8c61-4eeb-a2f4-38dd068f2172 with distance 0.9109553115041773.
